In [1]:
import mne
import os
from ieeg.viz.mri import plot_subj, plot_subj_sig_and_nonsig, _plot_electrodes, subject_to_info, plot_on_average
from ieeg.io import raw_from_layout, get_data
from misc_functions import load_sig_chans, channel_names_to_indices
import json
import pyvista as pv
pv.global_theme.allow_empty_mesh = True

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### plot single subject's significant electrodes

In [ ]:
subj = "D0100"
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus_fixationCrossBase_1sec_mirror.json'
stim_sig_chans = load_sig_chans(stim_filename)
# resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response_fixationCrossBase_1sec_mirror.json'
# resp_sig_chans = load_sig_chans(stim_filename)


subj_without_zeroes = "D" + subj[1:].lstrip('0')
brain_fig = plot_subj(subj_without_zeroes, picks=stim_sig_chans, labels_every=1, hemi='both', title=f'{subj}_Stimulus', background='black', color=(1, 1, 0)) 
# plot_subj(subj_without_zeroes, picks=resp_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Response', color=(1, 1, 0))

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_sigStim.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')


In [ ]:
subj = "D0063"
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus.json'
congruent_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_congruent.json'
incongruent_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_incongruent.json'
repeat_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_repeat.json'
switch_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_switch.json'

resp_sig_chans = load_sig_chans(resp_filename)
stim_sig_chans = load_sig_chans(stim_filename)
congruent_sig_chans = load_sig_chans(congruent_filename)
incongruent_sig_chans = load_sig_chans(incongruent_filename)
repeat_sig_chans = load_sig_chans(repeat_filename)
switch_sig_chans = load_sig_chans(switch_filename)


subj_without_zeroes = "D" + subj[1:].lstrip('0')
plot_subj(subj_without_zeroes, picks=stim_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Stimulus', background='black', color=(1, 1, 0)) 
plot_subj(subj_without_zeroes, picks=resp_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Response', background='black', color=(1, 1, 0))
plot_subj(subj_without_zeroes, picks=congruent_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Congruent', background='black', color=(1, 1, 0)) 
plot_subj(subj_without_zeroes, picks=incongruent_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Incongruent', background='black', color=(1, 1, 0))
plot_subj(subj_without_zeroes, picks=repeat_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Repeat', background='black', color=(1, 1, 0)) 
plot_subj(subj_without_zeroes, picks=switch_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Switch', background='black', color=(1, 1, 0))


load in lpfc sig elecs

In [7]:
# Specify the same filename
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
filename = os.path.join(save_dir, 'roi_dict_sig')

# Read the dictionary from the file
with open(filename, 'r') as f:
    sig_electrodes_per_subject_roi_loaded = json.load(f)

filename = os.path.join(save_dir, 'roi_dict_all')

# Read the dictionary from the file
with open(filename, 'r') as f:
    all_electrodes_per_subject_roi_loaded = json.load(f)

# Now you can use sig_electrodes_per_subject_roi_loaded as if it were the original dictionary
print(sig_electrodes_per_subject_roi_loaded['lpfc'])

{'D0057': ['RAI6', 'RPI14'], 'D0059': ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16'], 'D0063': ['LMSF5', 'LPSF12', 'RAI4', 'RAI6', 'RAMF12', 'RMMF13', 'RMMF14', 'RMMF10'], 'D0065': ['RASF14'], 'D0069': [], 'D0071': ['RFO14', 'RIA4', 'RIP6', 'RIA5', 'RIA11', 'RIA12', 'RIA14', 'RIA16'], 'D0077': [], 'D0090': ['RIA6', 'RIA12'], 'D0094': ['LFAM8', 'LFAM9', 'LFAM10', 'LFPM10', 'LFPM12', 'LFAI5', 'LPAI9', 'LFAI9', 'LFAI10', 'LIA11'], 'D0100': [], 'D0102': ['RFO13', 'RFO14', 'RFAM15'], 'D0103': ['LFAM8', 'LFAM9', 'LAI13', 'LAI14', 'LAI4', 'LFAI4']}


### plot all significant electrodes on average brain

In [ ]:
# grab all of the significant electrodes, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())


base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

subjects = layout.get(return_type="id", target="subject")
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]

allSigElectrodes = {}  # Initialize as an empty dictionary
allElectrodes = {}

for subj in subjects:
    # skip d0107 for now cuz i haven't preprocessed them yet 3/25
    if subj == "D0107":
        continue
    
    # resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_response_experimentStartBase1secTo101Sec.json'
    stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus_fixationCrossBase_1sec_mirror.json'

    # resp_sig_chans = load_sig_chans(resp_filename)
    stim_sig_chans = load_sig_chans(stim_filename)

    # Combine the significant electrodes for this subject
    # combined_chans = resp_sig_chans + stim_sig_chans
    combined_chans = stim_sig_chans # just plot stimulus for now, not response
    # Add them to the dictionary, tagged with the subject without zeroes

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    allSigElectrodes[subj_without_zeroes] = combined_chans

    # Get all electrodes for this subject and add them to the allElectrodes dict
    info = subject_to_info(subj_without_zeroes)
    allSubjElectrodes = mne.pick_channels(info.ch_names, [])
    allElectrodes[subj_without_zeroes] = allSubjElectrodes

picks = [f'D{k[1:].zfill(4)}-{v}' for k in allSigElectrodes.keys() for v in allSigElectrodes[k]]
# sinfos = [subject_to_info(f'D{int(subj[1:])}') for subj in subjects]

plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0), size=0.45)
plot_on_average(allElectrodes, hemi='both', color=(1,1,0))

In [4]:
allSigElectrodes

{'D57': ['RAI2',
  'RAI3',
  'RAI4',
  'RAI5',
  'RAI6',
  'RAI7',
  'RAI8',
  'RPI5',
  'RPI6',
  'RPI7',
  'RPI8',
  'RPI9',
  'RPI10',
  'RPI11',
  'RPI12',
  'RPI13',
  'RPI14',
  'RPI15',
  'RAIF7',
  'RAMF3',
  'RAMF4',
  'RAMF13',
  'RAMF14',
  'RMMF2',
  'RMMF3',
  'RMMF4',
  'RMMF5',
  'RMMF6',
  'RMMF7',
  'RMMF8',
  'RMMF9',
  'RMMF10',
  'RMMF11',
  'RMMF12',
  'RMMF13',
  'RMMF14',
  'RMMF15',
  'RMMF16',
  'RPIP5',
  'RPIP6',
  'RPIP7',
  'RPIP8',
  'RPIP9',
  'RPIP10',
  'RPIP11',
  'RPIP12',
  'RPIP13',
  'RPIP14',
  'RAST7',
  'RAST8',
  'RAIT1',
  'RAIT2',
  'RAIT3',
  'RAIT4',
  'RAIT5',
  'RAMT6',
  'RMMT6',
  'RPMT3',
  'RPMT4',
  'RPMT5',
  'RPIT3',
  'RPIT4',
  'RPIT5',
  'RPIT6',
  'RPIT7',
  'RPIT8',
  'RPIT9',
  'RPIT10',
  'RPIT12',
  'LAMT3',
  'LAMT4',
  'LAMT5',
  'LAMT6',
  'LAMT7',
  'LAMT8',
  'LAMT9',
  'LAMT10',
  'LAMT11',
  'LMMT2',
  'LMMT3',
  'LMMT4',
  'LMMT5',
  'LMMT6',
  'LMMT7',
  'LMMT8',
  'LMMT9',
  'LMMT10'],
 'D59': ['LMSF5',
  'LMSF6',

### plot all significant electrodes, then plot all electrodes

In [2]:
# grab all of the significant electrodes, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())


base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

subjects = layout.get(return_type="id", target="subject")

subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]

allSigElectrodes = {}  # Initialize as an empty dictionary
allElectrodes = {}

for subj in subjects:
    # if subj == "D0077":
    #     continue
    
    # resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
    stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus_fixationCrossBase_1sec_mirror.json'

    # resp_sig_chans = load_sig_chans(resp_filename)
    stim_sig_chans = load_sig_chans(stim_filename)

    # Combine the significant electrodes for this subject
    # combined_chans = resp_sig_chans + stim_sig_chans
    combined_chans = stim_sig_chans # delete this line once we have responses for everyone
    # Add them to the dictionary, tagged with the subject without zeroes

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    allSigElectrodes[subj_without_zeroes] = combined_chans

    # Get all electrodes for this subject and add them to the allElectrodes dict
    info = subject_to_info(subj_without_zeroes)
    allSubjElectrodes = mne.pick_channels(info.ch_names, [])
    allElectrodes[subj_without_zeroes] = allSubjElectrodes

# apparently this doesn't work because the inputs are dicts and plot on average doesn't like dict inputs.
# use Aaron's code below that converts from these dicts to a list of electrode indices 3/25. 
# fig = plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0), size=0.45)
# plot_on_average(allElectrodes, hemi='both', color=(1,1,1), fig=fig)

{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 12 | Sessions: 0 | Runs: 48}
KeysView({'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 12 | Sessions: 0 | Runs: 48})
Loaded significant channels for subject D0057


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


aaron's way of plotting

In [3]:
allSigElectrodes
subjects = []
length = 0
elecs = []
for subject, electrodes in allSigElectrodes.items():
    info = subject_to_info(subject)
    for electrode in electrodes:
        elecs.append(length + info['ch_names'].index(electrode))
    length += len(info['ch_names'])
    subjects.append(subject)

fig = plot_on_average(subjects, picks=elecs, rm_wm=False, hemi='both', color=(1, 1, 0), size=0.4, transparency=0.2)
# plot_on_average(subjects, rm_wm=False, hemi='both', color=(1, 1, 1), fig=fig)

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

Using pyvistaqt 3d backend.



C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

plot sig for congruency at any time window and also sig for interaction between congruency and congruency proportion at any time window 3/25  

this is for all electrodes

In [ ]:
LAB_root = None
channels = None
full_trial_base = False
task = 'GlobalLocal'
if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

congruency_congruencyProportion_structure_file_path = os.path.join(save_dir, 'congruency_congruencyProportion_ANOVAwithinElectrodes_significantEffectsStructure_wholeBrainAnalysis.txt')
with open(congruency_congruencyProportion_structure_file_path, 'r') as file:
    congruency_congruencyProportion_structure = json.load(file)

switchType_switchProportion_structure_file_path = os.path.join(save_dir, 'switchType_switchProportion_ANOVAwithinElectrodes_significantEffectsStructure_wholeBrainAnalysis.txt')
with open(switchType_switchProportion_structure_file_path, 'r') as file:
    switchType_switchProportion_structure = json.load(file)

congruency_switchType_structure_file_path = os.path.join(save_dir, 'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_wholeBrainAnalysis.txt')
with open(congruency_switchType_structure_file_path, 'r') as file:
    congruency_switchType_structure = json.load(file)

this is just for lpfc 4/1

In [6]:
LAB_root = None
channels = None
full_trial_base = False
task = 'GlobalLocal'
if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
roi = 'lpfc'

# congruency_congruencyProportion_structure_file_path = os.path.join(save_dir, 'congruency_congruencyProportion_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
# with open(congruency_congruencyProportion_structure_file_path, 'r') as file:
#     congruency_congruencyProportion_structure = json.load(file)

# switchType_switchProportion_structure_file_path = os.path.join(save_dir, 'switchType_switchProportion_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
# with open(switchType_switchProportion_structure_file_path, 'r') as file:
#     switchType_switchProportion_structure = json.load(file)

congruency_switchType_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(congruency_switchType_structure_file_path, 'r') as file:
    congruency_switchType_structure = json.load(file)

whole brain plotting setup (old as of 4/1, delete once the other method works for whole brain plotting)

In [ ]:
congruency_elecs = []
switchType_elecs = []
congruency_congruencyProportion_elecs = []
switchType_switchProportion_elecs = []
congruency_switchType_elecs = []

subjects = []
added_elecs_congruency = set()  # To keep track of added electrodes for congruency
added_elecs_congruency_congruencyProportion = set()  # To keep track of added electrodes for congruency_congruencyProportion

added_elecs_switchType = set()  # To keep track of added electrodes for switch type
added_elecs_switchType_switchProportion = set()  # To keep track of added electrodes for switchType_switchProportion

added_elecs_congruency_switchType = set()  # To keep track of added electrodes for congruency_switchType

def process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, effect_name, elecs_list, added_elecs_set):
    """A helper function to process electrode effects and update lists and sets accordingly."""
    for time_window, effects in time_windows.items():
        for effect_and_p_value in effects:
            if effect_and_p_value[0] == effect_name and (subject_without_zeroes, electrode) not in added_elecs_set:
                elecs_list.append(length + info['ch_names'].index(electrode))
                added_elecs_set.add((subject_without_zeroes, electrode))
                
# Assuming all structures have the same subjects...
length = 0
for subject in congruency_congruencyProportion_structure:
    subject_without_zeroes = "D" + subject[1:].lstrip('0')
    info = subject_to_info(subject_without_zeroes)

    # Ensure each subject is only added once
    if subject_without_zeroes not in subjects:
        subjects.append(subject_without_zeroes)

    current_subject_length = len(info['ch_names'])

    # Process congruency and congruency:congruencyProportion effects
    electrodes_dict_congruency_congruencyProportion = congruency_congruencyProportion_structure[subject]
    for electrode, time_windows in electrodes_dict_congruency_congruencyProportion.items():
        process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'congruency', congruency_elecs, added_elecs_congruency)
        process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'congruency:congruencyProportion', congruency_congruencyProportion_elecs, added_elecs_congruency_congruencyProportion)

    # Process switchType and switchType:switchProportion effects
    electrodes_dict_switchType_switchProportion = switchType_switchProportion_structure.get(subject, {})
    for electrode, time_windows in electrodes_dict_switchType_switchProportion.items():
        process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'switchType', switchType_elecs, added_elecs_switchType)
        process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'switchType:switchProportion', switchType_switchProportion_elecs, added_elecs_switchType_switchProportion)

    # Process congruency and switch type interaction effect
    electrodes_dict_congruency_switchType = congruency_switchType_structure.get(subject, {})
    for electrode, time_windows in electrodes_dict_congruency_switchType.items():
        process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'congruency:switchType', congruency_switchType_elecs, added_elecs_congruency_switchType)

    length += current_subject_length  # Update length after processing each subject

adapt whole brain setup so it works with roi plotting too 4/1

In [4]:
congruency_elecs

NameError: name 'congruency_elecs' is not defined

In [ ]:
added_elecs_congruency

In [48]:
# Assuming all structures have the same subjects...
subjects = layout.get(return_type="id", target="subject")
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103'] #remove this once i get d107 preprocessed
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]

congruency_elecs = []
switchType_elecs = []
congruency_congruencyProportion_elecs = []
switchType_switchProportion_elecs = []
congruency_switchType_elecs = []

added_elecs_congruency = set()  # To keep track of added electrodes for congruency
added_elecs_congruency_congruencyProportion = set()  # To keep track of added electrodes for congruency_congruencyProportion

added_elecs_switchType = set()  # To keep track of added electrodes for switch type
added_elecs_switchType_switchProportion = set()  # To keep track of added electrodes for switchType_switchProportion

added_elecs_congruency_switchType = set()  # To keep track of added electrodes for congruency_switchType

def process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, effect_name, elecs_list, added_elecs_set):
    """A helper function to process electrode effects and update lists and sets accordingly."""
    for time_window, effects in time_windows.items():
        for effect_and_p_value in effects:
            if effect_and_p_value[0] == effect_name and (subject_without_zeroes, electrode) not in added_elecs_set:
                elecs_list.append(length + info['ch_names'].index(electrode))
                added_elecs_set.add((subject_without_zeroes, electrode))
                
length = 0
ch_names = []
for subject in subjects:
    subject_without_zeroes = "D" + subject[1:].lstrip('0')
    info = subject_to_info(subject_without_zeroes)

    ch_names += ['-'.join([subject, n]) for n in info['ch_names']] #get a list of all channel names
    current_subject_length = len(info['ch_names'])

    # # Process congruency and congruency:congruencyProportion effects
    # electrodes_dict_congruency_congruencyProportion = congruency_congruencyProportion_structure[subject]
    # for electrode, time_windows in electrodes_dict_congruency_congruencyProportion.items():
    #     process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'congruency', congruency_elecs, added_elecs_congruency)
    #     process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'congruency:congruencyProportion', congruency_congruencyProportion_elecs, added_elecs_congruency_congruencyProportion)

    # # Process switchType and switchType:switchProportion effects
    # electrodes_dict_switchType_switchProportion = switchType_switchProportion_structure.get(subject, {})
    # for electrode, time_windows in electrodes_dict_switchType_switchProportion.items():
    #     process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'switchType', switchType_elecs, added_elecs_switchType)
    #     process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'switchType:switchProportion', switchType_switchProportion_elecs, added_elecs_switchType_switchProportion)

    # Process congruency and switch type interaction effect
    if congruency_switchType_structure and subject in congruency_switchType_structure:
        electrodes_dict_congruency_switchType = congruency_switchType_structure.get(subject, {})
        for electrode, time_windows in electrodes_dict_congruency_switchType.items():
            process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'congruency', congruency_elecs, added_elecs_congruency) # delete this line if i use congruency_congruencyProportion
            process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'switchType', switchType_elecs, added_elecs_switchType) # delete this line if i use switchType_switchProportion
            process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'congruency:switchType', congruency_switchType_elecs, added_elecs_congruency_switchType)

    length += current_subject_length  # Update length after processing each subject

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

plot unique congruency, unique congruency and congruency proportion interaction, and also their overlap

In [49]:
# Convert lists to sets for set operations
congruency_elecs_set = set(congruency_elecs)
congruency_congruencyProportion_elecs_set = set(congruency_congruencyProportion_elecs)

# Find overlapping electrodes (present in both lists)
overlap_congruencyEffectAndLWPC_elecs = congruency_elecs_set.intersection(congruency_congruencyProportion_elecs_set)

# Find unique electrodes in congruency_elecs
unique_congruency_elecs = congruency_elecs_set - congruency_congruencyProportion_elecs_set

# Find unique electrodes in congruency_congruencyProportion_elecs
unique_congruency_congruencyProportion_elecs = congruency_congruencyProportion_elecs_set - congruency_elecs_set

# Convert sets back to lists if necessary for plotting functions
overlap_congruencyEffectAndLWPC_elecs_list = list(overlap_congruencyEffectAndLWPC_elecs)
unique_congruency_elecs_list = list(unique_congruency_elecs)
unique_congruency_congruencyProportion_elecs_list = list(unique_congruency_congruencyProportion_elecs)

# Plotting
# Plot unique electrodes for congruency in one color
fig = plot_on_average(subjects, picks=unique_congruency_elecs_list, rm_wm=False, hemi='both', color=(1, 0, 0), size=0.35, label_every=1)

# Plot unique electrodes for congruency:congruencyProportion in another color
fig = plot_on_average(subjects, picks=unique_congruency_congruencyProportion_elecs_list, rm_wm=False, hemi='both', color=(0.5, 0, 0.5), size=0.35, fig=fig)

# Plot overlapping electrodes in black
fig = plot_on_average(subjects, picks=overlap_congruencyEffectAndLWPC_elecs_list, rm_wm=False, hemi='both', color=(0, 0, 0), size=0.35, fig=fig)

my_names = [ch_names[i] for i in unique_switchType_elecs]

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

plot switch type and switch type:switchProportion

In [51]:
# Convert lists to sets for set operations
congruency_elecs_set = set(congruency_elecs)

# Convert lists to sets for set operations
switchType_elecs_set = set(switchType_elecs)

switchType_switchProportion_elecs_set = set(switchType_switchProportion_elecs)

# Find overlapping electrodes (present in both lists)
overlap_switchCostAndLWPS_elecs = switchType_elecs_set.intersection(switchType_switchProportion_elecs_set)

# Find unique electrodes in switch type elecs
unique_switchType_elecs = switchType_elecs_set - switchType_switchProportion_elecs_set

# Find unique electrodes in congruency_congruencyProportion_elecs
unique_switchType_switchProportion_elecs = switchType_switchProportion_elecs_set - switchType_elecs_set

# Convert sets back to lists if necessary for plotting functions
overlap_switchCostAndLWPS_elecs_list = list(overlap_switchCostAndLWPS_elecs)
unique_switchType_elecs_list = list(unique_switchType_elecs)
unique_switchType_switchProportion_elecs_list = list(unique_switchType_switchProportion_elecs)

# Plotting
# Plot unique electrodes for switch type in one color
fig = plot_on_average(subjects, picks=unique_switchType_elecs_list, rm_wm=False, hemi='both', color=(0, 1, 0), size=0.35, label_every=1)

# Plot unique electrodes for switchType:switchProportion in another color
fig = plot_on_average(subjects, picks=unique_switchType_switchProportion_elecs_list, rm_wm=False, hemi='both', color=(0.6, 1, 0.6), size=0.35, fig=fig)

# Plot overlapping electrodes in black
fig = plot_on_average(subjects, picks=overlap_switchCostAndLWPS_elecs_list, rm_wm=False, hemi='both', color=(0, 0, 0), size=0.35, fig=fig)

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

plot congruency vs switch type

In [55]:
congruency_elecs_set = set(congruency_elecs)
switchType_elecs_set = set(switchType_elecs)
congruency_switchType_elecs_set = set(congruency_switchType_elecs)

# Find overlapping electrodes (present in both lists). Then also find union with the interaction effect.
overlap_congruencyEffectAndSwitchCost_elecs = switchType_elecs_set.intersection(congruency_elecs_set)
overlap_and_interaction_congruencyEffectAndSwitchCost_elecs = (overlap_congruencyEffectAndSwitchCost_elecs | congruency_switchType_elecs_set)

# Find unique electrodes in switchType_elecs
unique_switchType_elecs = switchType_elecs_set - congruency_elecs_set - congruency_switchType_elecs_set

# Find unique electrodes in congruency_elecs
unique_congruency_elecs = congruency_elecs_set - switchType_elecs_set - congruency_switchType_elecs_set

# Convert sets back to lists if necessary for plotting functions
overlap_congruencyEffectAndSwitchCost_elecs_list = list(overlap_congruencyEffectAndSwitchCost_elecs)
overlap_and_interaction_congruencyEffectAndSwitchCost_elecs_list = list(overlap_and_interaction_congruencyEffectAndSwitchCost_elecs)

unique_switchType_elecs_list = list(unique_switchType_elecs)
unique_congruency_elecs_list = list(unique_congruency_elecs)

#convert indices back to channel names because indices are broken for plotting right now 4/2. Just remove the indices step entirely later.
unique_switchType_elecs_names = [ch_names[i] for i in unique_switchType_elecs]
unique_congruency_elecs_names = [ch_names[i] for i in unique_congruency_elecs]
overlap_and_interaction_congruencyEffectAndSwitchCost_elecs_names = [ch_names[i] for i in overlap_and_interaction_congruencyEffectAndSwitchCost_elecs]

# Plotting
# Plot unique electrodes for congruency effect
fig = plot_on_average(subjects, picks=unique_congruency_elecs_names, rm_wm=False, hemi='both', color=(1, 0, 0), size=0.35, transparency=0.4)

# Plot unique electrodes for switchType in another color
fig = plot_on_average(subjects, picks=unique_switchType_elecs_names, rm_wm=False, hemi='both', color=(0, 1, 0), size=0.35, fig=fig)

# Plot overlapping electrodes and interaction electrodes in black
fig = plot_on_average(subjects, picks=overlap_and_interaction_congruencyEffectAndSwitchCost_elecs_names, rm_wm=False, hemi='both', color=(0, 0, 0), size=0.35, fig=fig)

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

In [29]:
unique_switchType_elecs_list

[962, 195, 981, 1527, 1464, 1465]

In [56]:
len(unique_congruency_elecs_list)

10

In [23]:
len(overlap_and_interaction_congruencyEffectAndSwitchCost_elecs_list)

14

plot congruency:switchType interaction vs intersection of congruency and switch type  
i am aware these variable names are awful and inconsistent. Change them later.. 3/27

In [39]:
congruency_switchType_elecs_set = set(congruency_switchType_elecs)

# Find overlapping electrodes (present in both lists)
overlap_congruency_switchType_and_overlap_congruencyEffectAndSwitchCost_elecs = congruency_switchType_elecs_set.intersection(overlap_congruencyEffectAndSwitchCost_elecs)

# Find unique electrodes in conSwitchInteraction
unique_congruency_switchType_elecs = congruency_switchType_elecs_set - overlap_congruencyEffectAndSwitchCost_elecs

# Find unique electrodes in overlap_congruencyEffectAndSwitchCost_elecs
unique_overlap_congruencyEffectAndSwitchCost_elecs = overlap_congruencyEffectAndSwitchCost_elecs - congruency_switchType_elecs_set

# Convert sets back to lists if necessary for plotting functions
overlap_congruency_switchType_and_overlap_congruencyEffectAndSwitchCost_elecs_list = list(overlap_congruency_switchType_and_overlap_congruencyEffectAndSwitchCost_elecs)
unique_congruency_switchType_elecs_list = list(unique_congruency_switchType_elecs)
unique_overlap_congruencyEffectAndSwitchCost_elecs_list = list(unique_overlap_congruencyEffectAndSwitchCost_elecs)

# Plotting
# Plot unique electrodes for con switch interaction
fig = plot_on_average(subjects, picks=unique_congruency_switchType_elecs_list, rm_wm=True, hemi='both', color=(0.5, 0.3, 0.1), size=0.35, transparency=0.2)

# Plot unique electrodes for con switch overlap
fig = plot_on_average(subjects, picks=unique_overlap_congruencyEffectAndSwitchCost_elecs_list, rm_wm=True, hemi='both', color=(1, 1, 1), size=0.35, fig=fig, transparency=0.2)

# Plot overlapping electrodes in black
fig = plot_on_average(subjects, picks=overlap_congruency_switchType_and_overlap_congruencyEffectAndSwitchCost_elecs_list, rm_wm=True, hemi='both', color=(0, 0, 0), size=0.35, fig=fig, transparency=0.2)

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

plot congruency switch interaction and additive effects VS congruency and switch type unique main effects

In [38]:
conSwitchInteractionOrAdditiveEffects = (set(congruency_switchType_interaction_elecs) | set(overlap_congruencyEffectAndSwitchCost_elecs))
conSwitchMainEffects = (set(congruency_elecs) | set(switchType_elecs))

# Find overlapping electrodes (present in both lists)
overlapMainAndInteractionEffects = conSwitchInteractionOrAdditiveEffects.intersection(conSwitchMainEffects)

# Find unique electrodes in conSwitchInteractionOrAdditiveEffects
unique_conSwitchInteractionOrAdditiveEffects_elecs = conSwitchInteractionOrAdditiveEffects - conSwitchMainEffects

# Find unique electrodes in conSwitchMainEffects
unique_conSwitchMainEffects_elecs = conSwitchMainEffects - conSwitchInteractionOrAdditiveEffects

# Convert sets back to lists if necessary for plotting functions
overlapMainAndInteractionEffects_list = list(overlapMainAndInteractionEffects)
unique_conSwitchInteractionOrAdditiveEffects_elecs_list = list(unique_conSwitchInteractionOrAdditiveEffects_elecs)
unique_conSwitchMainEffects_elecs_list = list(unique_conSwitchMainEffects_elecs)

# Plotting
# Plot unique electrodes for con switch interaction
fig = plot_on_average(subjects, picks=unique_conSwitchInteractionOrAdditiveEffects_elecs_list, rm_wm=False, hemi='both', color=(0, 1, 0), size=0.35, transparency=0.2)

# Plot unique electrodes for con switch overlap
fig = plot_on_average(subjects, picks=unique_conSwitchMainEffects_elecs_list, rm_wm=False, hemi='both', color=(1, 0, 0), size=0.35, fig=fig, transparency=0.2)

# Plot overlapping electrodes in black
fig = plot_on_average(subjects, picks=overlapMainAndInteractionEffects_list, rm_wm=False, hemi='both', color=(0, 0, 0), size=0.35, fig=fig, transparency=0.2)

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

In [ ]:
len(overlapMainAndInteractionEffects_list)

### plot all stim electrodes for an roi
get the subject: electrode mappings from roi_analysis first

In [ ]:
def get_sig_chans(sub, task, LAB_root=None):
    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    stim_filename = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
    stim_sig_chans = load_sig_chans(stim_filename)
    return stim_sig_chans


# List of subjects
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

# Initialize an empty dictionary to store significant channels per subject
sig_chans_per_subject = {}

# Populate the dictionary using get_sig_chans for each subject
for sub in subjects:
    sig_chans_per_subject[sub] = get_sig_chans(sub, 'GlobalLocal')

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

In [ ]:
import json

# Load from a JSON file
filename = 'subjects_electrodestoROIs_dict.json'

with open(filename, 'r') as file:
    subjects_electrodestoROIs_dict = json.load(file)

print(f"Loaded data from {filename}")

In [ ]:
def filter_electrodes_by_roi(subjects_electrodes_dict, sig_chans_per_subject, roi_list):
    """
    Filters electrodes based on specified ROIs and returns significant electrodes for each subject.

    Args:
    subjects_electrodes_dict (dict): A dictionary with subjects as keys and electrode-to-ROI mappings as values.
    sig_chans_per_subject (dict): A dictionary with subjects as keys and lists of significant channels as values.
    roi_list (list): A list of ROIs to filter electrodes.

    Returns:
    dict: A dictionary with subjects as keys and lists of significant electrodes in specified ROIs as values.
    """
    filtered_electrodes_per_subject = {}

    for sub, electrodes_dict in subjects_electrodes_dict.items():
        filtered = {key: value for key, value in electrodes_dict['filtROI_dict'].items() 
                    if any(roi in key for roi in roi_list)}

        # Aggregate electrodes into a list for each subject
        filtered_electrodes = []
        for electrodes in filtered.values():
            filtered_electrodes.extend(electrodes)

        filtered_electrodes_per_subject[sub] = filtered_electrodes
        print(f'For subject {sub}, {", ".join(roi_list)} electrodes are: {filtered_electrodes}')

    # Now filter for significant electrodes
    sig_filtered_electrodes_per_subject = {}

    for sub, filtered_electrodes in filtered_electrodes_per_subject.items():
        # Retrieve the list of significant channels for the subject
        sig_chans = sig_chans_per_subject.get(sub, [])

        # Find the intersection of filtered electrodes and significant channels for the subject
        sig_filtered_electrodes = [elec for elec in filtered_electrodes if elec in sig_chans]

        # Store the significant filtered electrodes for the subject
        sig_filtered_electrodes_per_subject[sub] = sig_filtered_electrodes
        print(f"Subject {sub} significant {', '.join(roi_list)} electrodes: {sig_filtered_electrodes}")

    return filtered_electrodes_per_subject, sig_filtered_electrodes_per_subject

# Example usage:
dlpfc_rois = ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"] #dorsolateral prefrontal cortex
acc_rois = ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"] #anterior cingulate cortex
parietal_rois = ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"] #superior parietal lobule, intraparietal sulcus, and inferior parietal lobule (split into angular gyrus and supramarginal gyrus)
dlpfc_electrodes_per_subject, sig_dlpfc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, dlpfc_rois)
acc_electrodes_per_subject, sig_acc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, acc_rois)
parietal_electrodes_per_subject, sig_parietal_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, parietal_rois)

this is with zeroes removed from subject names, i think plot on average needs those instead of the normal subject names  
also, remove the subjects with empty lists since those might be breaking the plotting too

In [ ]:
def remove_leading_zeroes_and_empty_subjects(subjects_dict):
    """
    Removes leading zeroes from subject names, adds a '_without_zeroes' suffix, 
    and filters out subjects with empty lists.

    Args:
    subjects_dict (dict): A dictionary with subjects as keys and lists as values.

    Returns:
    dict: A new dictionary with modified subject names and non-empty lists.
    """
    new_dict = {}
    for sub, value in subjects_dict.items():
        if value:  # Check if the list is not empty
            sub_without_zeroes = "D" + sub[1:].lstrip('0')
            new_dict[sub_without_zeroes] = value
    return new_dict

# Example usage:
dlpfc_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(dlpfc_electrodes_per_subject)
sig_dlpfc_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(sig_dlpfc_electrodes_per_subject)

acc_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(acc_electrodes_per_subject)
sig_acc_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(sig_acc_electrodes_per_subject)

parietal_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(parietal_electrodes_per_subject)
sig_parietal_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(sig_parietal_electrodes_per_subject)

### plot sig roi electrodes on individual brains

#### dlpfc

In [ ]:
for subj in sig_dlpfc_electrodes_without_zeroes:
    print(subj)

manually do single subjects

In [ ]:
subj = 'D0103'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

subj_without_zeroes = "D" + subj[1:].lstrip('0')
print("Picks for subject:", sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes])
brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=dlpfc_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_dlPFC', background='black')
# brain_fig = plot_subj(subj_without_zeroes, picks=sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes], labels_every=1, hemi='both', title=f'{subj}_dlPFC', background='black', color=(1, 1, 0))

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_dlPFC.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

loop thru everyone who has sig dlpfc electrodes

In [ ]:
for subj in sig_dlpfc_electrodes_without_zeroes:
    if subj == 'D57':
        continue #skip this one cuz its being weird
    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    print("Picks for subject:", sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes])

    brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=dlpfc_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_dlPFC', background='black')

    # Specify the path and filename for the PNG file
    png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_dlPFC.png")

    # Save the figure
    brain_fig.save_image(png_filepath, 'png')

#### acc

In [ ]:
for subj in sig_acc_electrodes_without_zeroes:
    print(subj)

manually do single subjects

In [ ]:
subj = 'D0103'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

subj_without_zeroes = "D" + subj[1:].lstrip('0')
print("Picks for subject:", sig_acc_electrodes_without_zeroes[subj_without_zeroes])

brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_acc_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=acc_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_acc', background='black')

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_acc.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

loop thru everyone who has sig acc electrodes

In [ ]:
for subj in sig_acc_electrodes_without_zeroes:

    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    print("Picks for subject:", sig_acc_electrodes_without_zeroes[subj_without_zeroes])

    brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_acc_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=acc_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_acc', background='black')

    # Specify the path and filename for the PNG file
    png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_acc.png")

    # Save the figure
    brain_fig.save_image(png_filepath, 'png')

#### parietal

In [ ]:
for subj in sig_parietal_electrodes_without_zeroes:
    print(subj)

manually do single subjects

In [ ]:
subj = 'D103'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

subj_without_zeroes = "D" + subj[1:].lstrip('0')
print("Picks for subject:", sig_parietal_electrodes_without_zeroes[subj_without_zeroes])

brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_parietal_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=parietal_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_parietal', background='black')

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_parietal.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

loop thru everyone who has sig parietal electrodes

In [ ]:
for subj in sig_parietal_electrodes_without_zeroes:

    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    print("Picks for subject:", sig_parietal_electrodes_without_zeroes[subj_without_zeroes])

    brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_parietal_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=parietal_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_parietal', background='black')

    # Specify the path and filename for the PNG file
    png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_parietal.png")

    # Save the figure
    brain_fig.save_image(png_filepath, 'png')

### do dlpfc, acc, parietal coverage for each person

get all electrodes across dlpfc, acc, and parietal and stick them into one dictionary so we can plot coverage in control regions for each subject

In [ ]:
allSigControlElectrodes = {key: sig_parietal_electrodes_without_zeroes.get(key, []) + 
                      sig_dlpfc_electrodes_without_zeroes.get(key, []) + 
                      sig_acc_electrodes_without_zeroes.get(key, [])
                 for key in set(sig_parietal_electrodes_without_zeroes) | 
                             set(sig_dlpfc_electrodes_without_zeroes) | 
                             set(sig_acc_electrodes_without_zeroes)}

allControlElectrodes = {key: parietal_electrodes_without_zeroes.get(key, []) + 
                      dlpfc_electrodes_without_zeroes.get(key, []) + 
                      acc_electrodes_without_zeroes.get(key, [])
                 for key in set(parietal_electrodes_without_zeroes) | 
                             set(dlpfc_electrodes_without_zeroes) | 
                             set(acc_electrodes_without_zeroes)}

subj = 'D63'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

subj_without_zeroes = "D" + subj[1:].lstrip('0')
brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=allSigControlElectrodes[subj_without_zeroes], nonsig_picks=allControlElectrodes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_allControlElectrodes', background='black')

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_allControlElectrodes.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

### try to plot all sig electrodes in an roi on an avg brain  
this is plotting like all electrodes wtf why

In [ ]:
plot_on_average(sig_parietal_electrodes_without_zeroes, hemi='both', color=(1,1,0))

### plot each subject one by one on the fsaverage brain

In [ ]:
# grab all of the significant electrodes, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())


base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

subjects = layout.get(return_type="id", target="subject")
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]
allSigElectrodes = {}  # Initialize as an empty dictionary
allElectrodes = {}

subj = 'D0071'
    
resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus.json'

resp_sig_chans = load_sig_chans(resp_filename)
stim_sig_chans = load_sig_chans(stim_filename)

# Combine the significant electrodes for this subject
combined_chans = resp_sig_chans + stim_sig_chans

# Add them to the dictionary, tagged with the subject without zeroes

subj_without_zeroes = "D" + subj[1:].lstrip('0')
allSigElectrodes[subj_without_zeroes] = combined_chans

# Get all electrodes for this subject and add them to the allElectrodes dict
info = subject_to_info(subj_without_zeroes)
allSubjElectrodes = mne.pick_channels(info.ch_names, [])
allElectrodes[subj_without_zeroes] = allSubjElectrodes


plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0))
plot_on_average(allElectrodes, hemi='both', color=(1,1,0))